## 🧱 1. Armado del Dataset

Se parte del dataset original de ventas sell-in.

Granularidad: `<product_id, periodo>`

Transformación:
- Agrupar por `product_id`, `periodo`
- Sumarizar `tn` (toneladas)


In [1]:
import pandas as pd

# Cargar el archivo de sell-in (ajustar el path si es necesario)
df_raw = pd.read_csv("sell-in.txt", delimiter='\t')

# Agrupar por product_id y periodo
df_agg = (
    df_raw.groupby(['product_id', 'periodo'], as_index=False)
           .agg({'tn': 'sum'})
           .sort_values(['product_id', 'periodo'])
)

df_agg.head()


,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132


In [2]:
df_agg.shape

(31243, 3)

## 🧮 2. Cálculo de la Clase (target)

Se crea un nuevo campo `clase` que representa `tn` en `periodo + 2`.

Notas:
- Para calcularlo se hace un merge desplazando dos períodos hacia atrás por `product_id`.
- Los períodos `201911` y `201912` quedan sin target (NaN).


In [3]:
# Asegurarse de que periodo sea tipo entero
df_agg['periodo'] = df_agg['periodo'].astype(int)

# Crear índice temporal: mes_abs (mes absoluto)
periodos_ordenados = sorted(df_agg['periodo'].unique())
map_periodo_to_mesabs = {p: i + 1 for i, p in enumerate(periodos_ordenados)}

# Agregar columna mes_abs
df_agg['mes_abs'] = df_agg['periodo'].map(map_periodo_to_mesabs)

# Ordenar correctamente por producto y tiempo
df_agg = df_agg.sort_values(['product_id', 'mes_abs'])

# Crear campo tn+2 (la clase) como tn desplazado -2 hacia adelante
df_agg['tn+2'] = df_agg.groupby('product_id')['tn'].shift(-2)

# Revisar el resultado
df_agg.head(40)


,product_id,periodo,tn,mes_abs,tn+2
0,20001,201701,934.77222,1,1303.35771
1,20001,201702,798.01620,2,1069.96130
2,20001,201703,1303.35771,3,1502.20132
3,20001,201704,1069.96130,4,1520.06539
4,20001,201705,1502.20132,5,1030.67391
5,20001,201706,1520.06539,6,1267.39462
6,20001,201707,1030.67391,7,1316.94604
7,20001,201708,1267.39462,8,1439.75563
8,20001,201709,1316.94604,9,1580.47401
9,20001,201710,1439.75563,10,1049.38860


## 🛠️ 3. Feature Engineering: Lags

Se generan 11 columnas de `tn` anteriores (tn_1 a tn_11), por `product_id`.

En cada fila, se tiene el historial de 12 meses completos si está disponible:
- `tn`, `tn_1`, ..., `tn_11`

No se generan features adicionales.


In [4]:
# Crear los lags tn_1 a tn_11
for lag in range(1, 12):
    df_agg[f'tn_{lag}'] = df_agg.groupby('product_id')['tn'].shift(lag)

df_agg.head(40)


,product_id,periodo,tn,mes_abs,tn+2,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
0,20001,201701,934.77222,1,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,798.01620,2,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201703,1303.35771,3,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201704,1069.96130,4,1520.06539,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201705,1502.20132,5,1030.67391,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,201706,1520.06539,6,1267.39462,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN
6,20001,201707,1030.67391,7,1316.94604,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN
7,20001,201708,1267.39462,8,1439.75563,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN
8,20001,201709,1316.94604,9,1580.47401,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN
9,20001,201710,1439.75563,10,1049.38860,1316.94604,1267.39462,1030.67391,1520.06539,1502.20132,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN


## 🎯 4. Dataset de Entrenamiento

Se selecciona únicamente el período `201812`.

Subset estratégico: solo los 33 `product_id` mágicos con datos completos.

Campos usados:
- Input: `tn`, `tn_1`, ..., `tn_11`
- Target: `clase` (mes `201902`)


In [5]:
# Lista de product_id mágicos
magicos = [20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
   20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
   20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
   20193, 20320, 20532, 20612, 20637, 20807, 20838]

# Selección de features y target
features = ['tn'] + [f'tn_{i}' for i in range(1, 12)]
target = 'tn+2'

# Filtrar el dataset para entrenamiento: mes_abs == 24 (equivale a periodo 201812)
df_train = df_agg[df_agg['mes_abs'] == 24].copy()

# Quedarse solo con los mágicos
df_train = df_train[df_train['product_id'].isin(magicos)]

# Eliminar registros incompletos
df_train = df_train.dropna(subset=features + [target])

# Mostrar resultados
print(f"Registros para entrenamiento: {df_train.shape[0]}")
display(df_train[features + [target]].head(10))


Registros para entrenamiento: 33


,tn,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn+2
23,1486.68669,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,1856.83534,1043.76470,1169.07532,1259.09363
59,1009.45458,1766.81068,1378.49032,954.23575,1161.88430,977.40239,1033.82845,1103.39191,999.20934,966.86044,712.00087,984.80167,1043.01349
95,769.82869,1206.91773,1313.34211,912.34156,955.97079,656.22700,660.73323,784.35885,765.47838,778.55594,788.30749,907.56304,758.32657
203,407.75925,566.66809,513.15472,478.04388,615.70617,515.20419,468.15260,865.28861,748.44391,862.19361,588.56272,470.33785,479.99914
275,426.32899,433.50170,532.45644,436.96269,554.82147,526.38149,554.57063,707.59267,691.53246,765.98901,506.25385,469.29224,476.98787
347,285.02947,414.97753,612.50721,480.60235,582.83104,331.96807,223.87746,227.24082,171.74107,653.77607,477.48363,298.25586,337.76009
383,321.09714,289.13976,177.75576,189.59850,191.07270,300.26178,437.75550,484.04538,562.70214,526.99374,601.26066,340.75314,431.62938
599,259.32724,286.83676,331.23254,288.35292,374.95908,351.60065,316.45841,533.53335,550.29417,488.79258,377.84497,291.70926,308.71060
635,326.01506,371.52958,161.58557,282.43485,375.61778,325.03223,420.33781,388.43687,543.06908,510.33171,337.54792,342.16945,265.84135
671,446.69747,532.98143,552.71975,417.95455,387.73155,351.05610,262.33076,356.42982,290.39581,321.26878,629.89543,243.71984,323.66178


## 📈 5. Entrenamiento del Modelo (Regresión Lineal)

Modelo: Regresión Lineal sin hiperparámetros

- X: tn, tn_1, ..., tn_11
- y: clase


In [6]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# Definir features y target
X = df_train[features]
y = df_train['tn+2']

# Entrenar modelo de regresión lineal
model = LinearRegression()
model.fit(X, y)

# Crear DataFrame de coeficientes
coef = pd.DataFrame({
    'feature': ['intercept'] + features,
    'coeficiente': [model.intercept_] + list(model.coef_)
})

# Ordenar por valor absoluto del coeficiente (opcional)
coef['abs'] = coef['coeficiente'].abs()
# coef = coef.sort_values(by='abs', ascending=False).drop(columns='abs').reset_index(drop=True)

# Mostrar coeficientes
display(coef)


,feature,coeficiente,abs
0,intercept,0.441467,0.441467
1,tn,-0.001339,0.001339
2,tn_1,0.236558,0.236558
3,tn_2,0.178208,0.178208
4,tn_3,-0.060031,0.060031
5,tn_4,-0.161875,0.161875
6,tn_5,-0.007775,0.007775
7,tn_6,0.151936,0.151936
8,tn_7,0.043933,0.043933
9,tn_8,0.142839,0.142839


## 📊 6. Aplicación del Modelo a los 780 registros finales

- Se aplicará solo a los 656 con datos completos.
- Los 124 restantes se imputan con el promedio.


In [7]:
# Agarramos los 780 productos correspondientes a predecir
df_780 = pd.read_csv("ListadoIDS.txt", sep=';', header=None)
df_780.columns = ['product_id']
df_780.shape

(781, 1)

In [10]:
df_agg['product_id'] = df_agg['product_id'].astype(str)
df_780['product_id'] = df_780['product_id'].astype(str)

In [11]:
import numpy as np

# --- FILTRADO BASE ---
# Solo productos a predecir y mes 201912 (mes_abs == 36)
df_pred = df_agg[
    (df_agg['mes_abs'] == 36) & 
    (df_agg['product_id'].isin(df_780['product_id']))
].copy()


# Determinar qué productos tienen todos los features disponibles
df_pred['completos'] = df_pred[features].notnull().all(axis=1)
df_pred.head (10)

,product_id,periodo,tn,mes_abs,tn+2,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,completos
35,20001,201912,1504.68856,36,NaN,1397.37231,1561.50552,1660.00561,1261.34529,1678.99318,1109.93769,1629.78233,1647.63848,1470.65653,1259.09363,1275.77351,True
71,20002,201912,1087.30855,36,NaN,1423.57739,1979.53635,1090.18771,813.78215,1066.44999,928.36431,1034.98927,1287.62346,1083.62552,1043.01349,1266.78751,True
107,20003,201912,892.50129,36,NaN,948.29393,1081.36645,967.77116,635.59563,715.20314,662.38654,590.12515,565.33774,638.04010,758.32657,964.76919,True
143,20004,201912,637.90002,36,NaN,723.94206,1064.69633,786.17140,482.13372,521.71519,667.19411,603.31081,466.70901,619.77084,441.70332,511.33713,True
179,20005,201912,593.24443,36,NaN,606.91173,996.78275,879.52808,536.66800,745.74978,876.39696,897.26297,624.99880,488.21387,409.89950,363.58438,True
215,20006,201912,417.23228,36,NaN,399.61420,528.32630,409.95501,262.73593,343.11053,458.04180,527.68846,835.47883,502.43741,479.99914,578.74461,True
251,20007,201912,390.43432,36,NaN,357.85913,445.34884,369.74894,307.82899,573.37257,536.13689,394.67651,511.54995,576.23305,368.79546,377.66902,True
287,20008,201912,195.36854,36,NaN,396.49833,452.77197,330.56343,233.00983,524.04994,567.42091,486.36682,403.69191,454.57037,476.98787,543.27828,True
323,20009,201912,495.03574,36,NaN,711.89025,556.15182,558.45719,520.41758,716.07987,610.39590,578.48564,391.28033,525.47182,366.72969,465.47521,True
359,20010,201912,359.59998,36,NaN,470.96658,448.82078,524.94628,199.86233,463.91662,600.25060,408.41680,446.72413,522.87583,337.76009,370.75591,True


In [ ]:
# Separar completos e incompletos
df_completos = df_pred[df_pred['completos']].copy()
df_incompletos = df_pred[~df_pred['completos']].copy()

# --- PREDICCIÓN PARA COMPLETOS ---
df_completos['pred'] = model.predict(df_completos[features])
df_completos['pred_tipo'] = 'modelo'

In [13]:
import numpy as np
# --- PREDICCIÓN PARA INCOMPLETOS: promedio ponderado con énfasis en febrero anterior (tn_10) ---

# Pesos manuales: últimos 3 meses, febrero pasado, resto
pesos = np.array([
    3.0,  # tn
    2.5,  # tn_1
    2.0,  # tn_2
    1.5,  # tn_3
    1.5,  # tn_4
    1.5,  # tn_5
    1.0,  # tn_6
    1.0,  # tn_7
    1.0,  # tn_8
    1.0,  # tn_9
    4.0,  # tn_10
    1.0   # tn_11
])

# Dar más peso a tn_10 (correspondiente a mes_abs 26 = 201902)
pesos[10] *= 12  # Aumentar influencia de febrero del año anterior

# Extraer matriz de features
X_incompletos = df_incompletos[features].values

# Crear máscara de valores válidos
máscara_validos = ~np.isnan(X_incompletos)

# Expandir pesos por fila, aplicando la máscara
pesos_expandido = np.tile(pesos, (X_incompletos.shape[0], 1))
pesos_validos = pesos_expandido * máscara_validos

# Calcular promedio ponderado por fila
suma_ponderada = np.nansum(X_incompletos * pesos_validos, axis=1)
suma_pesos = np.nansum(pesos_validos, axis=1)
df_incompletos['pred'] = suma_ponderada / suma_pesos
df_incompletos['pred_tipo'] = 'promedio_ponderado_febrero'


# --- UNIÓN FINAL ---
df_final = pd.concat([df_completos, df_incompletos], axis=0).sort_values('product_id')

# --- VALIDACIÓN DE RESULTADOS ---
print("🔍 Suma de TN a 201912:")
print(f"Completos (modelo): {df_completos['tn'].sum():,.2f}")
print(f"Incompletos (promedio historia): {df_incompletos['tn'].sum():,.2f}")
print(f"Total final: {df_final['tn'].sum():,.2f}")

print("🔍 Suma de predicciones a 202002:")
print(f"Completos (modelo): {df_completos['pred'].sum():,.2f}")
print(f"Incompletos (promedio historia): {df_incompletos['pred'].sum():,.2f}")
print(f"Total final: {df_final['pred'].sum():,.2f}")

print("\n📦 Desglose:")
print(f"Total a predecir: {df_final.shape[0]}  (esperado: {df_780.shape[0]})")
print(f"Completos: {df_completos.shape[0]}  |  Incompletos: {df_incompletos.shape[0]}")
print(df_final['pred_tipo'].value_counts())

# --- VISTA RÁPIDA ---
display(df_final[['product_id', 'periodo', 'pred', 'pred_tipo']].head())



🔍 Suma de TN a 201912:
Completos (modelo): 23,447.86
Incompletos (promedio historia): 1,697.39
Total final: 25,145.25
🔍 Suma de predicciones a 202002:
Completos (modelo): 25,785.02
Incompletos (promedio historia): 1,939.48
Total final: 27,724.50

📦 Desglose:
Total a predecir: 780  (esperado: 781)
Completos: 656  |  Incompletos: 124
pred_tipo
modelo                        656
promedio_ponderado_febrero    124
Name: count, dtype: int64


,product_id,periodo,pred,pred_tipo
35,20001,201912,1162.707525,modelo
71,20002,201912,1183.640604,modelo
107,20003,201912,684.763931,modelo
143,20004,201912,580.484961,modelo
179,20005,201912,563.560780,modelo


In [14]:
import os
from datetime import datetime

# Crear carpeta 'kaggle' si no existe
os.makedirs("kaggle", exist_ok=True)

# Generar timestamp actual
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Nombre de archivo
filename = f"kaggle/predicciones_201912_{timestamp}.csv"

# Exportar CSV
df_final[['product_id', 'pred']].to_csv(filename, index=False)

print(f"Archivo guardado como: {filename}")


Archivo guardado como: kaggle/predicciones_201912_20250704_1534.csv
